In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from scipy import stats
from scipy.stats.stats import pearsonr
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
plt.style.use('fivethirtyeight')

customers = pd.read_csv('data/olist_customers_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
products_translation = pd.read_csv('data/product_category_name_translation.csv')

In [8]:
orders_cleaned = orders[orders['order_status']!='canceled']
orders_cleaned['order_status'].value_counts()
products['volume'] = products['product_length_cm']*products['product_height_cm']*products['product_width_cm']
productsTranslated = pd.merge(products, products_translation, on='product_category_name', how='inner')
orders_cleaned['order_purchase_timestamp'] = pd.to_datetime(orders_cleaned['order_purchase_timestamp'])

/var/folders/rz/_nyqj12d6695z910qshyf3jr0000gn/T/ipykernel_28870/3494667942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_cleaned['order_purchase_timestamp'] = pd.to_datetime(orders_cleaned['order_purchase_timestamp'])


In [9]:
order_sales = pd.merge(orders_cleaned, order_payments, on='order_id', how='inner')
orderItems = pd.merge(orders_cleaned, order_items, on='order_id', how='inner')
order_products = pd.merge(orderItems, productsTranslated, on='product_id', how='inner')
order_products = order_products.rename(columns={'product_category_name_english':'product_category'})

In [16]:
order_products.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'volume', 'product_category'],
      dtype='object')

In [17]:
box_prop = order_products[['product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm']]
box_prop.corr()

,product_weight_g,product_length_cm,product_height_cm,product_width_cm
product_weight_g,1.000000,0.459961,0.583273,0.505394
product_length_cm,0.459961,1.000000,0.190362,0.531628
product_height_cm,0.583273,0.190362,1.000000,0.278501
product_width_cm,0.505394,0.531628,0.278501,1.000000


In [12]:
box_prop_scaled[] = StandardScaler().fit_transform(wine_raw_scaled)

aca2eb7d00ea1a7b8ebd4e68314663af    527
99a4788cb24856965c36a24e339b6058    487
422879e10f46682990de24d770e7f83d    484
389d119b48cf3043d311335e499d9c6b    391
368c6c730842d78016ad823897a372db    388
                                   ... 
a69e3d74b5056570954717a7efc806de      1
5ca6364f727e63db88d3d2aed16d7be8      1
ae23e5838bdd9b5fa518342c066b8290      1
e273c62ea30d5ca1753739e6d25d15a1      1
006619bbed68b000c8ba3f8725d5409e      1
Name: product_id, Length: 32124, dtype: int64